# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.data.dataset_factory import FileDatasetFactory
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from automl_data_utils import get_image_data_as_df, convert_to_table

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'covid-pneumonia-automl'

experiment=Experiment(ws, experiment_name)

compute_name = "compute-cluster1"
vm_size = "STANDARD_NC6"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print("Found existing cluster, using it!")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=6)
    compute_cluster = ComputeTarget.create(ws, compute_name, compute_config)
compute_cluster.wait_for_completion()

Found existing cluster, using it!


In [3]:
train_data, test_data = get_image_data_as_df(ws, './Covid19-dataset/train', './Covid19-dataset/test',"covid-pneumonia-dataset-table")

Found 66 images belonging to 3 classes.
Type: float64


In [4]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,12991,12992,12993,12994,12995,12996,12997,12998,12999,class
0,0.000000,0.007843,0.039216,0.066667,0.078431,0.101961,0.109804,0.117647,0.125490,0.211765,...,0.396078,0.349020,0.125490,0.074510,0.007843,0.000000,0.000000,0.000000,0.023529,2
1,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,...,0.250980,0.184314,0.066667,0.054902,0.054902,0.054902,0.054902,0.054902,0.054902,0
2,0.000000,0.000000,0.000000,0.952941,0.364706,0.207843,0.231373,0.266667,0.286275,0.588235,...,0.082353,0.082353,0.082353,0.082353,0.082353,0.082353,0.000000,0.000000,0.000000,1
3,0.090196,0.101961,0.145098,0.168627,0.203922,0.513726,0.384314,0.372549,0.376471,0.643137,...,0.000000,0.000000,0.000000,0.011765,0.019608,0.023529,0.015686,0.023529,0.027451,1
4,0.031373,0.039216,0.027451,0.031373,0.023529,0.031373,0.035294,0.019608,0.019608,0.031373,...,0.462745,0.188235,0.074510,0.015686,0.023529,0.035294,0.039216,0.039216,0.043137,1


In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_factory = TabularDatasetFactory.register_pandas_dataframe(train_data, target=ws.get_default_datastore(), name='CovidPneumoniaTable')

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/cd122184-bb30-4220-9254-f5949faad286/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {"enable_dnn":True,
                   "experiment_timeout_minutes":45,
                   "max_cores_per_iteration":3,
                   "enable_tf":False,
                   "featurization":"off"}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task='classification',
                             primary_metric='accuracy',
                             compute_target=compute_cluster,
                             training_data=data_factory,
                             label_column_name='class',
                             **automl_settings
                             )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
covid-pneumonia-automl,AutoML_e7d09855-477b-43a2-b49a-7ecf9a7f8aec,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_e7d09855-477b-43a2-b49a-7ecf9a7f8aec',
 'target': 'compute-cluster1',
 'status': 'Completed',
 'startTimeUtc': '2021-04-17T05:17:17.456526Z',
 'endTimeUtc': '2021-04-17T06:41:09.833316Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster1',
  'AMLSettingsJsonString': '{"path":null,"name":"covid-pneumonia-automl","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-142902","workspace_name":"quick-starts-ws-142902","region":"southcentralus","compute_target":"compute-cluster1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
#TODO: Save the best model
best_run = remote_run.get_best_child()
best_run_metrics = best_run.get_metrics()
best_run_parameters = best_run.get_details()
best_run_properties = best_run_parameters['properties']

print("Best Run:",best_run.id)
print("Algorithm:", best_run_properties['run_algorithm'])
print("Metric:", best_run_properties['primary_metric'])
print("Best Score:",best_run_properties['score'])


Best Run: AutoML_e7d09855-477b-43a2-b49a-7ecf9a7f8aec_12
Algorithm: VotingEnsemble
Metric: accuracy
Best Score: 0.9563076923076924


In [10]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_aac4c2448272f0d35fdeda99586e18654d96f92d5298773d19a499380b0ed435_d.txt',
 'azureml-logs/65_job_prep-tvmps_aac4c2448272f0d35fdeda99586e18654d96f92d5298773d19a499380b0ed435_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_aac4c2448272f0d35fdeda99586e18654d96f92d5298773d19a499380b0ed435_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/94_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service